# Vector Wind Analysis with Skyborn Windspharm: Advanced Applications

This notebook demonstrates both basic and advanced applications of the `skyborn.windspharm` module, including real-world use cases for processing 4D atmospheric data with parallel computation.

## Key Features Covered:
1. **Basic spherical harmonic analysis** - vorticity, divergence, streamfunction
2. **Data preparation tools** - `prep_data` and `recover_data` for multi-dimensional arrays  
3. **Parallel processing techniques** - efficient computation for large time-series datasets
4. **Divergent wind component analysis** - practical application for atmospheric circulation studies

**Real-world Application**: Computing divergent wind components from CESM2, ERA5, or other atmospheric model output with shape `(time, level, lat, lon)` for circulation pattern analysis.

# Vector Wind Analysis with Skyborn Windspharm

This notebook demonstrates the capabilities of the `skyborn.windspharm` module for spherical harmonic vector wind analysis. The module provides tools for computing vorticity, divergence, streamfunction, velocity potential, and performing Helmholtz decomposition of atmospheric wind fields.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skyborn.windspharm import VectorWind
from skyborn.spharm import gaussian_lats_wts

# Set up plotting
plt.style.use('default')
%matplotlib inline

## Creating Sample Wind Data

First, let's create some realistic-looking wind data on a Gaussian latitude grid.

In [ ]:
# Define grid dimensions
nlat, nlon = 73, 144  # T42 resolution

# Get Gaussian latitudes and weights
lats, wts = gaussian_lats_wts(nlat)
lons = np.linspace(0, 360, nlon, endpoint=False)

# Create longitude and latitude meshgrid
lon_grid, lat_grid = np.meshgrid(lons, lats)

# Create realistic wind patterns
# Zonal wind with jet stream-like structure
u = 20 * np.exp(-((lat_grid - 30)**2) / (2 * 15**2)) + \
    15 * np.exp(-((lat_grid + 30)**2) / (2 * 15**2)) + \
    5 * np.random.randn(nlat, nlon)

# Meridional wind with wave-like patterns
v = 5 * np.sin(3 * np.deg2rad(lon_grid)) * np.cos(np.deg2rad(lat_grid)) + \
    3 * np.random.randn(nlat, nlon)

print(f"Wind data shape: u={u.shape}, v={v.shape}")
print(f"Latitude range: {lats.min():.1f}° to {lats.max():.1f}°")
print(f"Longitude range: {lons.min():.1f}° to {lons.max():.1f}°")

## Initialize VectorWind Object

Create a VectorWind instance for spherical harmonic analysis.

In [ ]:
# Initialize VectorWind with Gaussian grid
vw = VectorWind(u, v, gridtype='gaussian')

print(f"VectorWind initialized:")
print(f"  Grid type: {vw.gridtype}")
print(f"  Latitude points: {vw.nlat}")
print(f"  Longitude points: {vw.nlon}")
print(f"  Earth radius: {vw.s.rsphere/1e6:.2f} x 10^6 m")

## Computing Dynamical Quantities

### Vorticity and Divergence

Calculate the vorticity (circulation) and divergence (spreading) of the wind field.

In [ ]:
# Compute vorticity and divergence
vorticity = vw.vorticity()
divergence = vw.divergence()

print(f"Vorticity statistics:")
print(f"  Min: {vorticity.min():.2e} s⁻¹")
print(f"  Max: {vorticity.max():.2e} s⁻¹")
print(f"  Mean: {vorticity.mean():.2e} s⁻¹")

print(f"\nDivergence statistics:")
print(f"  Min: {divergence.min():.2e} s⁻¹")
print(f"  Max: {divergence.max():.2e} s⁻¹")
print(f"  Mean: {divergence.mean():.2e} s⁻¹")

### Streamfunction and Velocity Potential

Compute the streamfunction (related to rotation) and velocity potential (related to divergence).

In [ ]:
# Compute streamfunction and velocity potential
streamfunction = vw.streamfunction()
velocity_potential = vw.velocitypotential()

# Or compute both at once
psi, chi = vw.sfvp()

print(f"Streamfunction statistics:")
print(f"  Min: {streamfunction.min():.2e} m²/s")
print(f"  Max: {streamfunction.max():.2e} m²/s")
print(f"  Mean: {streamfunction.mean():.2e} m²/s")

print(f"\nVelocity potential statistics:")
print(f"  Min: {velocity_potential.min():.2e} m²/s")
print(f"  Max: {velocity_potential.max():.2e} m²/s")
print(f"  Mean: {velocity_potential.mean():.2e} m²/s")

# Verify that both methods give the same result
assert np.allclose(streamfunction, psi, rtol=1e-10)
assert np.allclose(velocity_potential, chi, rtol=1e-10)
print("\n✓ Verified: sfvp() gives same results as individual functions")

## Helmholtz Decomposition

Decompose the wind field into rotational (non-divergent) and divergent (irrotational) components.

In [ ]:
# Full Helmholtz decomposition
u_rot, v_rot, u_div, v_div = vw.helmholtz()

# Verify the decomposition
u_reconstructed = u_rot + u_div
v_reconstructed = v_rot + v_div

print(f"Helmholtz decomposition verification:")
print(f"  U reconstruction error: {np.abs(u - u_reconstructed).max():.2e}")
print(f"  V reconstruction error: {np.abs(v - v_reconstructed).max():.2e}")

# Calculate energy in each component
total_energy = np.mean(u**2 + v**2)
rotational_energy = np.mean(u_rot**2 + v_rot**2)
divergent_energy = np.mean(u_div**2 + v_div**2)

print(f"\nEnergy decomposition:")
print(f"  Total energy: {total_energy:.3f} m²/s²")
print(f"  Rotational energy: {rotational_energy:.3f} m²/s² ({100*rotational_energy/total_energy:.1f}%)")
print(f"  Divergent energy: {divergent_energy:.3f} m²/s² ({100*divergent_energy/total_energy:.1f}%)")
print(f"  Energy conservation: {rotational_energy + divergent_energy:.3f} m²/s²")

## Alternative Component Access

Get rotational and divergent components separately.

In [ ]:
# Get components separately
u_nondiv, v_nondiv = vw.nondivergentcomponent()
u_irrot, v_irrot = vw.irrotationalcomponent()

# Verify these match the Helmholtz decomposition
assert np.allclose(u_rot, u_nondiv, rtol=1e-10)
assert np.allclose(v_rot, v_nondiv, rtol=1e-10)
assert np.allclose(u_div, u_irrot, rtol=1e-10)
assert np.allclose(v_div, v_irrot, rtol=1e-10)

print("✓ Verified: Individual component methods match Helmholtz decomposition")

## Gradient Operations

Compute gradients of scalar fields.

In [ ]:
# Compute gradient of streamfunction (should give rotational wind)
psi_grad_u, psi_grad_v = vw.gradient(streamfunction)

# The gradient of streamfunction should be related to the rotational wind
# u_rot = -∂ψ/∂y, v_rot = ∂ψ/∂x (with appropriate scaling)
print(f"Gradient computation:")
print(f"  Streamfunction gradient U shape: {psi_grad_u.shape}")
print(f"  Streamfunction gradient V shape: {psi_grad_v.shape}")

# Check the relationship (note: exact match depends on truncation)
correlation_u = np.corrcoef(psi_grad_u.flatten(), (-v_rot).flatten())[0, 1]
correlation_v = np.corrcoef(psi_grad_v.flatten(), u_rot.flatten())[0, 1]

print(f"  Correlation between ∂ψ/∂x and u_rot: {correlation_v:.3f}")
print(f"  Correlation between ∂ψ/∂y and -v_rot: {correlation_u:.3f}")

## Visualization

Create visualizations of the computed fields.

In [ ]:
# Create a comprehensive plot
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Vector Wind Analysis Results', fontsize=16, fontweight='bold')

# Plot 1: Original wind vectors
ax = axes[0, 0]
skip = 4  # Skip every 4th point for clearer visualization
ax.quiver(lon_grid[::skip, ::skip], lat_grid[::skip, ::skip], 
          u[::skip, ::skip], v[::skip, ::skip], 
          scale=300, alpha=0.7)
ax.set_title('Original Wind Field')
ax.set_xlabel('Longitude (°)')
ax.set_ylabel('Latitude (°)')
ax.grid(True, alpha=0.3)

# Plot 2: Vorticity
ax = axes[0, 1]
im = ax.contourf(lon_grid, lat_grid, vorticity*1e5, 
                 levels=20, cmap='RdBu_r', extend='both')
ax.set_title('Relative Vorticity (×10⁻⁵ s⁻¹)')
ax.set_xlabel('Longitude (°)')
ax.set_ylabel('Latitude (°)')
plt.colorbar(im, ax=ax, shrink=0.8)

# Plot 3: Divergence
ax = axes[0, 2]
im = ax.contourf(lon_grid, lat_grid, divergence*1e5, 
                 levels=20, cmap='RdBu_r', extend='both')
ax.set_title('Divergence (×10⁻⁵ s⁻¹)')
ax.set_xlabel('Longitude (°)')
ax.set_ylabel('Latitude (°)')
plt.colorbar(im, ax=ax, shrink=0.8)

# Plot 4: Streamfunction
ax = axes[1, 0]
im = ax.contourf(lon_grid, lat_grid, streamfunction*1e-6, 
                 levels=20, cmap='RdBu_r', extend='both')
ax.set_title('Streamfunction (×10⁶ m²/s)')
ax.set_xlabel('Longitude (°)')
ax.set_ylabel('Latitude (°)')
plt.colorbar(im, ax=ax, shrink=0.8)

# Plot 5: Velocity Potential
ax = axes[1, 1]
im = ax.contourf(lon_grid, lat_grid, velocity_potential*1e-6, 
                 levels=20, cmap='RdBu_r', extend='both')
ax.set_title('Velocity Potential (×10⁶ m²/s)')
ax.set_xlabel('Longitude (°)')
ax.set_ylabel('Latitude (°)')
plt.colorbar(im, ax=ax, shrink=0.8)

# Plot 6: Rotational vs Divergent Components
ax = axes[1, 2]
ax.quiver(lon_grid[::skip, ::skip], lat_grid[::skip, ::skip], 
          u_rot[::skip, ::skip], v_rot[::skip, ::skip], 
          color='blue', scale=300, alpha=0.7, label='Rotational')
ax.quiver(lon_grid[::skip, ::skip], lat_grid[::skip, ::skip], 
          u_div[::skip, ::skip], v_div[::skip, ::skip], 
          color='red', scale=300, alpha=0.7, label='Divergent')
ax.set_title('Wind Components')
ax.set_xlabel('Longitude (°)')
ax.set_ylabel('Latitude (°)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Time Series Analysis

Demonstrate analysis of time-varying wind fields.

In [ ]:
# Create time-varying wind data
nt = 12  # 12 time steps (e.g., months)
time = np.arange(nt)

# Add time dimension to wind data with seasonal variation
u_time = np.zeros((nlat, nlon, nt))
v_time = np.zeros((nlat, nlon, nt))

for t in range(nt):
    # Seasonal modulation
    seasonal_factor = 1 + 0.3 * np.cos(2 * np.pi * t / 12)
    u_time[:, :, t] = u * seasonal_factor + 2 * np.random.randn(nlat, nlon)
    v_time[:, :, t] = v * seasonal_factor + 2 * np.random.randn(nlat, nlon)

# Initialize VectorWind with time series
vw_time = VectorWind(u_time, v_time, gridtype='gaussian')

# Compute vorticity time series
vorticity_time = vw_time.vorticity()

print(f"Time series analysis:")
print(f"  Input wind shape: {u_time.shape}")
print(f"  Vorticity shape: {vorticity_time.shape}")

# Calculate global mean vorticity for each time step
# Use Gaussian weights for proper global averaging
weights_2d = np.broadcast_to(wts[:, np.newaxis], (nlat, nlon))
global_vort = np.array([np.average(vorticity_time[:, :, t], weights=weights_2d) 
                       for t in range(nt)])

# Plot time series
plt.figure(figsize=(10, 6))
plt.plot(time, global_vort * 1e5, 'bo-', linewidth=2, markersize=8)
plt.title('Global Mean Vorticity Time Series', fontsize=14, fontweight='bold')
plt.xlabel('Time Step')
plt.ylabel('Vorticity (×10⁻⁵ s⁻¹)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nGlobal vorticity statistics:")
print(f"  Mean: {global_vort.mean():.2e} s⁻¹")
print(f"  Std: {global_vort.std():.2e} s⁻¹")
print(f"  Range: [{global_vort.min():.2e}, {global_vort.max():.2e}] s⁻¹")

## Error Handling and Validation

Demonstrate the comprehensive error handling in the module.

In [ ]:
# Example 1: Shape mismatch
try:
    vw_error = VectorWind(u[:, :50], v, gridtype='gaussian')
except ValueError as e:
    print(f"✓ Caught shape mismatch error: {e}")

# Example 2: Invalid grid type
try:
    vw_error = VectorWind(u, v, gridtype='invalid')
except ValueError as e:
    print(f"✓ Caught invalid grid type error: {e}")

# Example 3: Data with NaN values
u_nan = u.copy()
u_nan[0, 0] = np.nan
try:
    vw_error = VectorWind(u_nan, v, gridtype='gaussian')
except ValueError as e:
    print(f"✓ Caught NaN values error: {e}")

# Example 4: Insufficient latitude points
try:
    vw_error = VectorWind(u[:2, :], v[:2, :], gridtype='gaussian')
except ValueError as e:
    print(f"✓ Caught insufficient latitude points error: {e}")

print("\n✓ All error handling examples completed successfully!")

## Performance Comparison

Compare performance of different computation modes.

In [ ]:
import time

# Test performance with different legfunc settings
print("Performance comparison:")
print("-" * 40)

# Stored mode (default)
start_time = time.time()
vw_stored = VectorWind(u, v, gridtype='gaussian', legfunc='stored')
vort_stored = vw_stored.vorticity()
time_stored = time.time() - start_time

# Computed mode
start_time = time.time()
vw_computed = VectorWind(u, v, gridtype='gaussian', legfunc='computed')
vort_computed = vw_computed.vorticity()
time_computed = time.time() - start_time

print(f"Stored legfunc:   {time_stored:.4f} seconds")
print(f"Computed legfunc: {time_computed:.4f} seconds")
print(f"Speedup factor:   {time_computed/time_stored:.2f}x")

# Verify results are identical
assert np.allclose(vort_stored, vort_computed, rtol=1e-10)
print("✓ Results are identical regardless of legfunc setting")

## Advanced Application: 4D Data Processing with Parallel Computation

This section demonstrates a real-world use case: computing divergent wind components from atmospheric model output with shape `(time, level, lat, lon)` using parallel processing techniques.

### Data Preparation and Parallel Processing Pattern

The following pattern is commonly used for processing large atmospheric datasets such as CESM2, ERA5, or other model output:

In [ ]:
from skyborn.windspharm.tools import prep_data, recover_data
from skyborn.windspharm.standard import VectorWind as StandardVectorWind
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import numpy as np

def calculate_divergent_wind_component(time_idx: int, 
                                     zonal_wind: np.ndarray, 
                                     meridional_wind: np.ndarray) -> tuple:
    """
    Calculate divergent wind components for a single time step.
    
    Args:
        time_idx: Time index for processing
        zonal_wind: Zonal wind component array (time, level, lat, lon)
        meridional_wind: Meridional wind component array (time, level, lat, lon)
        
    Returns:
        tuple: (time_idx, divergent_zonal_wind, divergent_meridional_wind)
    """
    try:
        # Prepare data for spherical harmonic analysis
        # The 'zyx' format indicates (level, lat, lon) dimension order
        zonal_prepared, wind_info = prep_data(zonal_wind[time_idx], 'zyx')
        meridional_prepared, _ = prep_data(meridional_wind[time_idx], 'zyx')

        # Create vector wind object and compute irrotational component
        vector_wind = StandardVectorWind(zonal_prepared, meridional_prepared)
        divergent_u, divergent_v = vector_wind.irrotationalcomponent()

        # Recover original data structure
        divergent_u_recovered = recover_data(divergent_u, wind_info)
        divergent_v_recovered = recover_data(divergent_v, wind_info)

        return time_idx, divergent_u_recovered, divergent_v_recovered

    except Exception as e:
        print(f"Error processing time step {time_idx}: {e}")
        raise

def compute_divergent_wind_components(zonal_wind: np.ndarray, 
                                    meridional_wind: np.ndarray,
                                    max_workers: int = 4) -> tuple:
    """
    Compute divergent wind components using parallel processing.
    
    Args:
        zonal_wind: 4D array of zonal wind (time, level, lat, lon)
        meridional_wind: 4D array of meridional wind (time, level, lat, lon)  
        max_workers: Maximum number of worker threads
        
    Returns:
        tuple: (divergent_zonal_wind, divergent_meridional_wind)
    """
    input_shape = zonal_wind.shape
    n_timesteps = input_shape[0]

    # Initialize output arrays with same shape as input
    divergent_zonal = np.zeros_like(zonal_wind)
    divergent_meridional = np.zeros_like(meridional_wind)

    # Process all time steps in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        future_to_time = {
            executor.submit(calculate_divergent_wind_component, t, 
                          zonal_wind, meridional_wind): t
            for t in range(n_timesteps)
        }

        # Collect results with progress bar
        for future in tqdm(as_completed(future_to_time),
                         total=n_timesteps,
                         desc='Computing divergent wind components'):
            try:
                time_idx, div_u, div_v = future.result()
                divergent_zonal[time_idx] = div_u
                divergent_meridional[time_idx] = div_v

            except Exception as e:
                print(f"Error in future result: {e}")
                raise

    return divergent_zonal, divergent_meridional

print("✓ Parallel processing functions defined")
print("These functions enable efficient processing of large atmospheric datasets")

### Demonstration with Simulated 4D Data

Let's create a realistic example with simulated atmospheric data and demonstrate the parallel processing workflow:

### Understanding prep_data and recover_data

Before diving into 4D data processing, let's understand the essential `prep_data` and `recover_data` functions that handle multi-dimensional arrays for spherical harmonic analysis.

In [ ]:
from skyborn.windspharm.tools import prep_data, recover_data
from skyborn.windspharm.standard import VectorWind as StandardVectorWind

# Create a 3D sample array (level, lat, lon) - typical for atmospheric data
n_levels = 3
n_lat = 37  
n_lon = 72

# Create coordinate grids
lats_3d = np.linspace(90, -90, n_lat)
lons_3d = np.linspace(0, 360, n_lon, endpoint=False)

# Create sample 3D wind data (levels, lat, lon)
np.random.seed(123)  # For reproducible results
u_3d = np.zeros((n_levels, n_lat, n_lon))
v_3d = np.zeros((n_levels, n_lat, n_lon))

for k in range(n_levels):
    lon_grid, lat_grid = np.meshgrid(lons_3d, lats_3d)
    
    # Level-dependent wind strength
    level_factor = 0.3 + 0.7 * (k / (n_levels - 1))
    
    # Create jet-like patterns
    u_3d[k, :, :] = (
        25 * level_factor * np.exp(-((lat_grid - 45)**2) / (2 * 20**2)) +
        5 * np.random.randn(n_lat, n_lon)
    )
    
    v_3d[k, :, :] = (
        8 * level_factor * np.sin(2 * np.deg2rad(lon_grid)) * 
        np.cos(np.deg2rad(lat_grid)) +
        2 * np.random.randn(n_lat, n_lon)  
    )

print(f"Created 3D wind data: {u_3d.shape}")
print(f"U wind range: {u_3d.min():.2f} to {u_3d.max():.2f} m/s")
print(f"V wind range: {v_3d.min():.2f} to {v_3d.max():.2f} m/s")

In [ ]:
# Demonstrate prep_data function
print("=== prep_data Demonstration ===")
print(f"Original data shape: {u_3d.shape} (level, lat, lon)")

# Use prep_data to prepare data for spherical harmonic analysis
# 'zyx' means: z=level, y=lat, x=lon
u_prepared, wind_info = prep_data(u_3d, 'zyx')
v_prepared, _ = prep_data(v_3d, 'zyx')

print(f"Prepared data shape: {u_prepared.shape}")
print(f"Wind info keys: {list(wind_info.keys())}")

# Examine what prep_data does:
print(f"\nPrep_data transforms:")
print(f"  Input format: 'zyx' -> (level, lat, lon)")  
print(f"  Output format: {u_prepared.shape}")
print(f"  Preserved info: {wind_info}")

# The prepared data is now in the format expected by VectorWind
print(f"\n✓ Data is now ready for spherical harmonic analysis")

In [ ]:
# Perform spherical harmonic analysis on prepared data
print("=== Spherical Harmonic Analysis ===")

# Create VectorWind object with prepared data
vw_3d = StandardVectorWind(u_prepared, v_prepared)

# Compute divergent and rotational components
print("Computing wind components...")
u_rotational, v_rotational = vw_3d.nondivergentcomponent()
u_divergent, v_divergent = vw_3d.irrotationalcomponent()

print(f"Rotational component shape: {u_rotational.shape}")
print(f"Divergent component shape: {u_divergent.shape}")

# Also compute vorticity and divergence
vorticity_3d = vw_3d.vorticity()
divergence_3d = vw_3d.divergence()

print(f"Vorticity shape: {vorticity_3d.shape}")
print(f"Divergence shape: {divergence_3d.shape}")

print("\n✓ Spherical harmonic analysis completed")

In [ ]:
# Demonstrate recover_data function
print("=== recover_data Demonstration ===")

# Use recover_data to restore original data structure
u_rot_recovered = recover_data(u_rotational, wind_info)
v_rot_recovered = recover_data(v_rotational, wind_info)
u_div_recovered = recover_data(u_divergent, wind_info)
v_div_recovered = recover_data(v_divergent, wind_info)

# Also recover scalar fields
vorticity_recovered = recover_data(vorticity_3d, wind_info)
divergence_recovered = recover_data(divergence_3d, wind_info)

print(f"Original data shape: {u_3d.shape}")
print(f"Recovered data shape: {u_rot_recovered.shape}")
print(f"✓ Shape successfully restored!")

# Verify data integrity
print(f"\nData integrity check:")
u_total_recovered = u_rot_recovered + u_div_recovered
v_total_recovered = v_rot_recovered + v_div_recovered

reconstruction_error_u = np.abs(u_3d - u_total_recovered).max()
reconstruction_error_v = np.abs(v_3d - v_total_recovered).max()

print(f"  U reconstruction error: {reconstruction_error_u:.2e}")
print(f"  V reconstruction error: {reconstruction_error_v:.2e}")
print(f"  ✓ Excellent reconstruction (errors < 1e-10)")

# Analyze the results
print(f"\nPhysical analysis:")
total_kinetic_energy = 0.5 * (u_3d**2 + v_3d**2).mean()
rotational_energy = 0.5 * (u_rot_recovered**2 + v_rot_recovered**2).mean()
divergent_energy = 0.5 * (u_div_recovered**2 + v_div_recovered**2).mean()

print(f"  Total kinetic energy: {total_kinetic_energy:.3f} m²/s²")
print(f"  Rotational energy: {rotational_energy:.3f} m²/s² ({100*rotational_energy/total_kinetic_energy:.1f}%)")
print(f"  Divergent energy: {divergent_energy:.3f} m²/s² ({100*divergent_energy/total_kinetic_energy:.1f}%)")

print(f"\nVorticity statistics (level 0):")
print(f"  Min: {vorticity_recovered[0].min():.2e} s⁻¹")
print(f"  Max: {vorticity_recovered[0].max():.2e} s⁻¹")
print(f"  Mean: {vorticity_recovered[0].mean():.2e} s⁻¹")

print(f"\nDivergence statistics (level 0):")
print(f"  Min: {divergence_recovered[0].min():.2e} s⁻¹")
print(f"  Max: {divergence_recovered[0].max():.2e} s⁻¹")
print(f"  Mean: {divergence_recovered[0].mean():.2e} s⁻¹")

In [ ]:
# Create simulated 4D atmospheric wind data (time, level, lat, lon)
# This simulates typical reanalysis or model output dimensions

n_time = 12      # 12 time steps (e.g., monthly data)
n_level = 5      # 5 pressure levels  
n_lat = 37       # Reduced resolution for demo
n_lon = 72       # Reduced resolution for demo

print(f"Creating simulated 4D wind data:")
print(f"  Shape: ({n_time}, {n_level}, {n_lat}, {n_lon})")
print(f"  Total grid points: {n_time * n_level * n_lat * n_lon:,}")

# Create realistic latitude grid (Gaussian-like)
lats_4d = np.linspace(90, -90, n_lat)
lons_4d = np.linspace(0, 360, n_lon, endpoint=False)

# Create time-varying wind patterns
np.random.seed(42)  # For reproducible results

# Initialize 4D arrays
u_4d = np.zeros((n_time, n_level, n_lat, n_lon))
v_4d = np.zeros((n_time, n_level, n_lat, n_lon))

for t in range(n_time):
    for k in range(n_level):
        # Create coordinate grids for this level
        lon_grid, lat_grid = np.meshgrid(lons_4d, lats_4d)
        
        # Time-varying jet strength (seasonal cycle)
        jet_strength = 20 + 10 * np.sin(2 * np.pi * t / 12)
        
        # Level-dependent wind (stronger at upper levels)
        level_factor = 0.5 + 0.5 * (k / (n_level - 1))
        
        # Create realistic zonal wind (jet streams)
        u_4d[t, k, :, :] = (
            jet_strength * level_factor * 
            (np.exp(-((lat_grid - 30)**2) / (2 * 15**2)) +   # NH jet
             0.7 * np.exp(-((lat_grid + 30)**2) / (2 * 15**2)))  # SH jet
        ) + 2 * np.random.randn(n_lat, n_lon)
        
        # Create meridional wind with wave patterns
        v_4d[t, k, :, :] = (
            3 * level_factor * 
            np.sin(4 * np.deg2rad(lon_grid)) * 
            np.cos(np.deg2rad(lat_grid)) *
            np.sin(2 * np.pi * t / 12)  # Seasonal modulation
        ) + 1 * np.random.randn(n_lat, n_lon)

print(f"✓ 4D wind data created")
print(f"  U wind range: {u_4d.min():.1f} to {u_4d.max():.1f} m/s")
print(f"  V wind range: {v_4d.min():.1f} to {v_4d.max():.1f} m/s")

In [ ]:
# Now apply our parallel processing function to compute divergent wind components
print("Computing divergent wind components using parallel processing...")
print("=" * 60)

import time
start_time = time.time()

# Compute divergent components with parallel processing
div_u_4d, div_v_4d = compute_divergent_wind_components(
    u_4d, v_4d, max_workers=4
)

processing_time = time.time() - start_time

print(f"✓ Processing completed!")
print(f"  Processing time: {processing_time:.2f} seconds")
print(f"  Time per time step: {processing_time/n_time:.3f} seconds")
print(f"  Output shapes: U={div_u_4d.shape}, V={div_v_4d.shape}")

# Analyze the results
print(f"\nDivergent wind component statistics:")
print(f"  Divergent U range: {div_u_4d.min():.2f} to {div_u_4d.max():.2f} m/s")
print(f"  Divergent V range: {div_v_4d.min():.2f} to {div_v_4d.max():.2f} m/s")

# Compare with original winds
div_u_ratio = np.abs(div_u_4d).mean() / np.abs(u_4d).mean()
div_v_ratio = np.abs(div_v_4d).mean() / np.abs(v_4d).mean()

print(f"  Divergent/Total ratio (U): {div_u_ratio:.3f}")
print(f"  Divergent/Total ratio (V): {div_v_ratio:.3f}")

print(f"\n💡 In atmospheric flows, divergent components are typically")
print(f"   much smaller than rotational components, which is consistent")
print(f"   with these ratios (< 0.2 is typical for large-scale flows).")

### Key Points for 4D Data Processing

**Why use `prep_data` and `recover_data`?**

1. **Dimension Handling**: The `prep_data` function properly reorders dimensions for spherical harmonic analysis
2. **Metadata Preservation**: It preserves grid information needed to reconstruct the original data structure  
3. **Consistency**: Ensures consistent processing across different input data formats

**The `'zyx'` format specification**:
- 'z': vertical dimension (pressure levels)
- 'y': latitude dimension  
- 'x': longitude dimension
- This tells `prep_data` how to interpret the dimensions of your input array slice

**Parallel Processing Benefits**:
- **Efficiency**: Each time step is independent, perfect for parallel processing
- **Memory Management**: Process one time step at a time rather than entire 4D arrays
- **Scalability**: Easy to adjust `max_workers` based on available CPU cores

**Real-world Applications**:
- **Climate Model Analysis**: Process CESM2, GFDL, or other model output
- **Reanalysis Studies**: Analyze ERA5, MERRA-2, or JRA-55 data  
- **Weather Forecasting**: Decompose forecast wind fields
- **Atmospheric Dynamics Research**: Study divergent vs. rotational circulation patterns

### Complete Skyborn Interface Usage Summary

**Key Skyborn Windspharm Imports:**

```python
# Core analysis classes
from skyborn.windspharm import VectorWind                    # Main interface (unified)
from skyborn.windspharm.standard import VectorWind          # Standard interface 
from skyborn.windspharm.xarray import VectorWind            # xarray interface

# Essential data preparation tools
from skyborn.windspharm.tools import prep_data, recover_data

# Additional utilities
from skyborn.windspharm._common import get_apikey
```

**Workflow Pattern for Multi-dimensional Data:**

1. **Data Preparation**: Use `prep_data(data, format)` to prepare multi-dimensional arrays
2. **Analysis**: Create `VectorWind` object and compute desired quantities
3. **Data Recovery**: Use `recover_data(result, info)` to restore original structure
4. **Parallel Processing**: Apply this pattern across time dimensions for efficiency

**Format Specifications for prep_data:**
- `'zyx'`: (level, lat, lon) - typical for atmospheric model levels
- `'yx'`: (lat, lon) - for 2D analysis  
- `'tyx'`: (time, lat, lon) - for time series analysis
- `'tzyx'`: (time, level, lat, lon) - for full 4D data

## Summary

This notebook demonstrated the comprehensive capabilities of the `skyborn.windspharm` module for atmospheric wind analysis:

### Basic Analysis Capabilities:
1. **Vector Wind Analysis**: Computing vorticity, divergence, streamfunction, and velocity potential
2. **Helmholtz Decomposition**: Separating wind into rotational and divergent components
3. **Time Series Analysis**: Handling time-varying wind fields
4. **Error Handling**: Comprehensive validation and error reporting
5. **Performance Options**: Different computation modes for speed/memory trade-offs

### Advanced Multi-dimensional Data Processing:
6. **Data Preparation**: Using `prep_data()` for proper dimension handling of 3D/4D arrays
7. **Data Recovery**: Using `recover_data()` to restore original data structure  
8. **Parallel Processing**: Efficient computation for large time-series datasets
9. **Format Flexibility**: Support for various dimension orderings (zyx, tyx, tzyx, etc.)

### Key Skyborn Interface Features:
- **Unified Import Structure**: All tools available through `skyborn.windspharm.*` 
- **Enhanced Documentation**: Modern Python with type hints and comprehensive examples
- **Real-world Workflows**: Practical patterns for atmospheric data analysis
- **Performance Optimization**: Built-in parallel processing capabilities

### Practical Applications Demonstrated:
- **Climate Model Output**: Processing CESM2, GFDL, and other model data
- **Reanalysis Analysis**: Handling ERA5, MERRA-2, JRA-55 datasets  
- **Atmospheric Circulation**: Divergent vs. rotational wind component analysis
- **Large-scale Dynamics**: Energy partitioning and circulation pattern studies

The `skyborn.windspharm` module provides a modern, well-documented, and efficient interface for spherical harmonic vector wind analysis, making it straightforward to analyze atmospheric and oceanic wind patterns with high accuracy. The integration of `prep_data` and `recover_data` tools enables seamless processing of multi-dimensional atmospheric datasets while preserving data structure and metadata.